In [1]:
import joblib
import numpy as np
import pandas as pd
from pathlib import Path


In [2]:

# === Paths ===
model_path = Path("C:/Users/jonat/fraud-detection-project/models/trained_model_manual_fields/mlp_manual_model.pkl")
preprocessor_path = Path("C:/Users/jonat/fraud-detection-project/models/trained_model_manual_fields/preprocessor_manual.pkl")
freq_map_path = Path("C:/Users/jonat/fraud-detection-project/data/processed/frequency_maps.pkl")


In [3]:

# === Load model, preprocessor, and frequency maps ===
model = joblib.load(model_path)
preprocessor = joblib.load(preprocessor_path)
frequency_maps = joblib.load(freq_map_path)

product_freq_map = frequency_maps["ProductCD"]
email_freq_map = frequency_maps["P_emaildomain"]


In [4]:

# === Sample raw input (simulate one from frontend) ===
raw_input = {
    "TransactionAmt": 100.0,
    "card1": 4444,
    "addr1": 300,
    "dist1": 15.2,
    "ProductCD": "W",
    "P_emaildomain": "gmail.com"
}


In [5]:

# === Map raw to frequency-encoded inputs ===
mapped_input = {
    "TransactionAmt": raw_input["TransactionAmt"],
    "card1": raw_input["card1"],
    "addr1": raw_input["addr1"],
    "dist1": raw_input["dist1"],
    "ProductCD_freq": product_freq_map.get(raw_input["ProductCD"], 0),
    "P_emaildomain_freq": email_freq_map.get(raw_input["P_emaildomain"].lower(), 0)
}


In [6]:

# === Create DataFrame and preprocess ===
input_df = pd.DataFrame([mapped_input])
input_processed = preprocessor.transform(input_df)

print("=== Mapped Input ===")
print(input_df)
print("\n=== Transformed Input ===")
print(input_processed)
print("Shape:", input_processed.shape)


=== Mapped Input ===
   TransactionAmt  card1  addr1  dist1  ProductCD_freq  P_emaildomain_freq
0           100.0   4444    300   15.2        0.744522            0.386688

=== Transformed Input ===
[[1.05338054e+02 4.73693522e+03 3.33295129e+02 1.66760922e+01
  9.66493709e-01 3.10438431e-01]]
Shape: (1, 6)


In [7]:

# === Run Prediction ===
prob = model.predict_proba(input_processed)[0][1]
pred = model.predict(input_processed)[0]

print("\n=== Prediction Result ===")
print("Prediction (0=legit, 1=fraud):", int(pred))
print("Fraud Probability:", round(prob, 4))



=== Prediction Result ===
Prediction (0=legit, 1=fraud): 0
Fraud Probability: 0.0
